# Level 2 - Week 7 - 01 Planning Patterns

**Estimated time:** 60-90 minutes

## Learning Objectives

- Define a fixed plan template
- Compare fixed vs replanning
- Set plan budgets


## Overview

Plans can be:

- too shallow (miss steps)
- too deep (waste tokens/time)

You need depth control.

## Underlying theory: planning is search under a budget

Planning chooses a sequence of actions that reduces uncertainty until you can answer.

- actions are tool calls or “write final answer”
- each action has a cost (latency, tokens, risk)

If a plan has steps $a_1, \ldots, a_T$, you can think in terms of a simple budgeted objective:

$$
\max_{a_{1:T}} \; \mathrm{Utility}(a_{1:T}) \quad \text{s.t.} \quad \mathrm{Cost}(a_{1:T}) \le B
$$

Deeper planning increases cost; shallow planning increases failure probability.

## Decomposition intuition

Decompose the task:

- what must I know to answer?
- which tool can provide that?
- what is the next unknown after I observe tool output?

## Patterns

- Fixed plan: plan once then execute
- Replanning: update after tool outputs
- Budgeted planning: cap plan length and steps

## Practice Steps

- Write a fixed plan template.
- Add a replanning rule (e.g. if search returns empty hits → clarify/refuse).
- Add budgets (max plan steps, max tool calls).

### Sample code

Fixed plan template and replanning hook.


In [ ]:
FIXED_PLAN = [
    'identify needed info',
    'call tools',
    'synthesize answer',
    'self-check',
]

print(FIXED_PLAN)


### Student fill-in

Write a plan for a sample task.


In [ ]:
def search_stub(query: str) -> list[str]:
    if "unknown" in query:
        return []
    return ["kb#001", "kb#002"]


def fixed_plan_for_task(task: str) -> list[str]:
    return [
        "identify_needed_info",
        "search",
        "write_answer",
        "self_check",
    ]


def replan_after_search(task: str, hits: list[str]) -> list[str]:
    if not hits:
        return ["clarify_user"]
    return ["write_answer", "self_check"]


task_ok = "summarize refund policy"
task_empty = "unknown policy"

for task in [task_ok, task_empty]:
    plan = fixed_plan_for_task(task)
    print("task:", task)
    print("fixed_plan:", plan)

    hits = search_stub(task)
    remaining = replan_after_search(task, hits)
    print("search_hits:", hits)
    print("replanned_remaining:", remaining)
    print("---")

## Self-check

- Do you cap plan length?
- Is replanning triggered by empty hits?


### Exercise: Budgeted planning + early stopping

Define explicit budgets:

- `max_plan_steps`
- `max_tool_calls`

If a budget is hit, switch to a safe behavior:

- ask the user for the single missing piece, or
- return a best-effort partial result + what is missing.

### Student fill-in

Implement a budget check that stops execution if:

- plan length exceeds `max_plan_steps`, or
- tool calls exceed `max_tool_calls`

Then test it on:

- an easy task that should finish within budget
- a task that triggers the budget and forces a safe stop

In [ ]:
def run_plan_with_budgets(task: str, max_plan_steps: int, max_tool_calls: int) -> dict:
    plan = fixed_plan_for_task(task)
    if len(plan) > max_plan_steps:
        return {"mode": "clarify", "reason": "plan_too_long", "plan": plan[:max_plan_steps]}

    tool_calls = 0
    hits: list[str] = []

    for step in plan:
        if step == "search":
            tool_calls += 1
            if tool_calls > max_tool_calls:
                return {"mode": "clarify", "reason": "tool_budget_hit", "tool_calls": tool_calls}
            hits = search_stub(task)
            if not hits:
                return {"mode": "clarify", "reason": "empty_hits"}

        if step == "write_answer":
            tool_calls += 1
            if tool_calls > max_tool_calls:
                return {"mode": "clarify", "reason": "tool_budget_hit", "tool_calls": tool_calls}

    return {"mode": "answer", "reason": "completed", "tool_calls": tool_calls, "hits": hits}


print(run_plan_with_budgets("summarize refund policy", max_plan_steps=6, max_tool_calls=3))
print(run_plan_with_budgets("unknown policy", max_plan_steps=6, max_tool_calls=3))
print(run_plan_with_budgets("summarize refund policy", max_plan_steps=2, max_tool_calls=3))

## Self-check

- Does your agent stop after N steps/tool calls?
- Does it switch to `clarify` when `search` returns empty hits?
- Can you explain why a plan was cut short (budget hit vs empty hits)?